In [21]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage import color
import matplotlib.pyplot as plt
from skimage.segmentation import slic, mark_boundaries
from sklearn.preprocessing import StandardScaler
import os

# Create directory for output results
output_dir = "/mnt/data/soil_analysis_results"
os.makedirs(output_dir, exist_ok=True)

def extract_texture_features(image_path, n_segments=100):
    """Extracts texture features from a soil image using LBP and GLCM."""
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Unable to load the image from {image_path}")
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    # Local Binary Pattern (LBP)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray, n_points, radius, method="uniform")
    
    # Gray Level Co-occurrence Matrix (GLCM)
    glcm = greycomatrix(gray, distances=[1], angles=[0], symmetric=True, normed=True)
    contrast = greycoprops(glcm, prop='contrast')[0, 0]
    homogeneity = greycoprops(glcm, prop='homogeneity')[0, 0]
    energy = greycoprops(glcm, prop='energy')[0, 0]
    correlation = greycoprops(glcm, prop='correlation')[0, 0]
    
    # Segment the image
    segments = slic(img_rgb, n_segments=n_segments, compactness=10, sigma=1)
    
    n_segments = np.max(segments) + 1
    segment_features = []
    
    # Collect LBP and GLCM features for each segment
    for i in range(n_segments):
        mask = segments == i
        if np.sum(mask) > 0:
            # LBP feature extraction
            lbp_segment = lbp[mask]
            lbp_mean = np.mean(lbp_segment)
            lbp_std = np.std(lbp_segment)
            
            # GLCM features
            texture_features = [contrast, homogeneity, energy, correlation]
            
            segment_features.append([lbp_mean, lbp_std] + texture_features)
    
    segment_features = np.array(segment_features)
    
    # Calculate the mean of the segment features
    texture_feature_mean = np.mean(segment_features, axis=0)
    
    return texture_feature_mean

# Load 5 reference soil samples with different types
sample_paths = ["black_soil.jpg", "cinder_soil.jpg", "laterite_soil.jpg", "peat_soil.jpg", "yellow_soil.jpg"]
sample_features = np.array([extract_texture_features(path) for path in sample_paths])

# Normalize features
scaler = StandardScaler()
normalized_samples = scaler.fit_transform(sample_features)

# Define soil types
soil_types = ["Black Soil", "Cinder Soil", "Laterite Soil", "Peat Soil", "Yellow Soil"]

def classify_soil(image_path):
    """Classifies soil based on texture features and compares with samples."""
    test_features = extract_texture_features(image_path)
    test_features = scaler.transform([test_features])
    distances = np.linalg.norm(normalized_samples - test_features, axis=1)
    closest_sample = np.argmin(distances)
    
    classified_soil = soil_types[closest_sample]
    
    # Determine Organic Matter level based on texture
    lbp_mean = test_features[0][0]
    if lbp_mean < 5:  # Threshold for determining High vs Low Organic Matter (Example)
        organic_matter = "Low"
    else:
        organic_matter = "High"
    
    print(f"Classified as: {classified_soil}")
    print(f"Organic Matter Level: {organic_matter}")
    return classified_soil, organic_matter

# Test with a new image
image_path = "test_soil4.jpg"
soil_classification, organic_matter_level = classify_soil(image_path)


ImportError: cannot import name 'greycomatrix' from 'skimage.feature' (c:\Users\HP\anaconda3\Lib\site-packages\skimage\feature\__init__.py)